In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import chess

from env import *
env = Chess_env()
obs = env.reset()

from deepQ import *

env.board

In [ ]:
for step in range(2):
    action = np.random.choice(list(env.board.legal_moves))
    _,_,done,_ = env.step(action)

env.board

In [ ]:
list(env.board.legal_moves)[10:]

In [ ]:
model = DeepQ(env, dropout_rate=0.1, n_channels=100, n_residual=1)
model.model.summary()

In [ ]:
%%time
model.predict_move_to_play_MCTS(5, env,400,)

In [ ]:
model.train(
    max_epoch = 1000, 
    epsilon_random_frames = 5000,
    epsilon_greedy_frames = 50000,
    max_memory_length = 5000,
    update_after_actions = 4,
    update_target_network = 1000,
    epsilon = 1,
    epsilon_min = 0.1,
    gamma = 0.99,
    batch_size = 32,
    max_steps_per_episode = 100,
    learning_rate=1e-4,
    MCTS_depth=8,
    MCTS_iterations=400
    )